In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
import os

loader = PyPDFLoader("files/中華郵政股份有限公司_資訊整體架構健檢及優化服務專案_建議書徵求說明書0420.pdf")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

# The Neo4jVector Module will connect to Neo4j and create a vector index if needed.

URL = os.getenv("URL")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

db = Neo4jVector.from_documents(
    docs, OpenAIEmbeddings(), url=URL, username=USERNAME, password=PASSWORD
)

query = "交通部中華郵政股份有限公司有什麼概述"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = """Based on the {context}, answer the following question: {question}

answer:
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser
answer = chain.invoke({"context":docs_with_score[0][0].page_content, "question":"交通部中華郵政股份有限公司有什麼概述"})

In [18]:
answer

'交通部中華郵政股份有限公司在驗收準則中設定了文件驗證準則和驗收審查準則，以確保得標廠商交付的文件和程式符合要求。公司要求文件和程式經審查後無嚴重缺失或中度缺失，並且輕度缺失已經矯正完畢，才能進行驗收。若有未完成的文件或程式缺失，廠商需在30天內完成矯正並經公司確認。此外，公司也會針對廠商交付的文件報告召開審查會議，確認其合理性和可用性。如果提出改善建議和方案，公司也會進行可行性和效益評估，並進行審查。'